## Intcode

In [1]:
# states are
# 0 - running
# 1 - waiting for input
# 99 - halt

mutable struct Prog
    code::Vector{Int}
    cur::Int
    input::Vector{Int}
    output::Vector{Int}
    relative_base::Int
    state::Int
end

str2prog(s) = parse.(Int, split(s, ","))

Prog(s::String) = Prog(vcat(str2prog(s), zeros(Int, 10000)), 1, [], [], 0, 0)

struct Instruction
    op::Int
    modes::Vector{Int}
end

function Instruction(op_code::Int)
    ops = Dict{Int, Int}(
        1 => 3,
        2 => 3,
        3 => 1,
        4 => 1,
        5 => 2,
        6 => 2,
        7 => 3,
        8 => 3,
        9 => 1,
        99 => 0
    )
    
    op = mod(op_code, 100)
    modes_code = div(op_code, 100)
    modes = zeros(ops[op])
    for i in 1:ops[op]
        modes[i] = mod(modes_code, 10)
        modes_code = div(modes_code, 10)
    end
    
    Instruction(op, modes)
end

Instruction(prog::Prog) = Instruction(prog.code[prog.cur])

function take(prog::Prog, mode, offset)
    if mode == 0
        return prog.code[prog.code[prog.cur + offset] + 1]
    elseif mode == 1
        return prog.code[prog.cur + offset]
    else
        return prog.code[prog.code[prog.cur + offset] + prog.relative_base + 1]
    end
end

function update!(prog::Prog, value, offset, mode = 0)
    if mode == 0
        prog.code[prog.code[prog.cur + offset] + 1] = value
    elseif mode == 2
        prog.code[prog.code[prog.cur + offset] + prog.relative_base + 1] = value
    end
end

function apply(prog::Prog, instruction::Instruction)
    if instruction.op == 99
        prog.state = 99
    elseif instruction.op == 1
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 + a2, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 2
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 * a2, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 3
        if isempty(prog.input)
            prog.state = 1
        else
            update!(prog, popfirst!(prog.input), 1, instruction.modes[1])
            prog.cur += 2
        end
    elseif instruction.op == 4
        push!(prog.output, take(prog, instruction.modes[1], 1))
        prog.cur += 2
    elseif instruction.op == 5
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        prog.cur = a1 != 0 ? a2 + 1 : prog.cur + 3
    elseif instruction.op == 6
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        prog.cur = a1 == 0 ? a2 + 1 : prog.cur + 3
    elseif instruction.op == 7
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 < a2 ? 1 : 0, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 8
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 == a2 ? 1 : 0, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 9
        prog.relative_base += take(prog, instruction.modes[1], 1)
        prog.cur += 2
    end
end

function run(prog::Prog, input::Vector{Int})
    prog.input = input
    prog.state = 0
    run(prog)
end

run(prog::Prog, input::Int) = run(prog, [input])

function run(prog)
    clear!(prog)
    while prog.state == 0
        instruction = Instruction(prog)
        apply(prog, instruction)
    end
    
    prog.output
end

function feed(prog::Prog, input::Vector{Int})
    prog.input = vcat(prog.input, input)
    prog.state = 0
end

clear!(prog) = (prog.output = [])

clear! (generic function with 1 method)

In [2]:
vm = Prog(readline("input.txt"))

Prog([109, 2050, 21101, 966, 0, 1, 21101, 0, 13, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1, Int64[], Int64[], 0, 0)

In [18]:
s = "NOT A T
NOT T J
NOT D T
NOT T T
AND T J
NOT B T
AND B J
WALK
"

s = "NOT D T
NOT T J
WALK
"

s = "NOT C J
NOT A T
OR T J
AND D J
WALK
"

vm = Prog(readline("input.txt"))
out = run(vm, Int.(collect(s)))
println(join(Char.(out[1:(end-1)])))

Input instructions:

Walking...




In [19]:
out[end]

19354173

## Part 2

In [27]:
s = "NOT A T
NOT T J
NOT D T
NOT T T
AND T J
NOT B T
AND B J
WALK
"

s = "NOT D T
NOT T J
WALK
"

s = "NOT C J
NOT A T
OR T J
AND D J
RUN
"

s = "NOT C J
NOT B T
OR T J
NOT A T
OR T J
AND D J
RUN
"

vm = Prog(readline("input.txt"))
out = run(vm, Int.(collect(s)))
println(join(Char.(out)))

Input instructions:

Running...


Didn't make it across:

.................
.................
@................
#####.#.#...#.###

.................
.................
.@...............
#####.#.#...#.###

.................
.................
..@..............
#####.#.#...#.###

.................
...@.............
.................
#####.#.#...#.###

....@............
.................
.................
#####.#.#...#.###

.................
.....@...........
.................
#####.#.#...#.###

.................
.................
......@..........
#####.#.#...#.###

.................
.................
.................
#####.#@#...#.###




In [54]:
function sensor_command(sensor, i)
    if i == 2
        return "NOT $sensor J\n"
    elseif i == 1
        return "NOT $sensor T\nNOT T J\n"
    elseif i == 4
        return "AND $sensor J\n"
    elseif i == 3
        return "NOT $sensor T\nAND T J\n"
    elseif i == 6
        return "OR $sensor J\n"
    else
        return "NOT $sensor T\nOR T J\n"
    end
end

function floodfill(sensors = collect('A':'D'))
    start = (Tuple{Char, Int}[], "")
    acc = [start]
    while !isempty(acc)
        s0, command = popfirst!(acc)
        used = [x[1] for x in s0]
        lng = isempty(s0) ? 0 : sum([mod(x[2], 2) + 1 for x in s0])
        avail = filter(x -> !(x in used), sensors)
        for sensor in avail
            for i in 1:6
                if lng + mod(i, 2) > 19 continue end
                new_command = command*sensor_command(sensor, i)*"WALK\n"
                println(new_command)
                vm = Prog(readline("input.txt"))
                out = run(vm, Int.(collect(new_command)))
                if out[end] > 1000 return out, new_command end
                push!(acc, (vcat(s0, [(sensor, i)]), command*sensor_command(sensor, i)))
            end
        end
    end
end

floodfill (generic function with 2 methods)

In [55]:
floodfill()

NOT A T
NOT T J
WALK

NOT A J
WALK

NOT A T
AND T J
WALK

AND A J
WALK

NOT A T
OR T J
WALK

OR A J
WALK

NOT B T
NOT T J
WALK

NOT B J
WALK

NOT B T
AND T J
WALK

AND B J
WALK

NOT B T
OR T J
WALK

OR B J
WALK

NOT C T
NOT T J
WALK

NOT C J
WALK

NOT C T
AND T J
WALK

AND C J
WALK

NOT C T
OR T J
WALK

OR C J
WALK

NOT D T
NOT T J
WALK

NOT D J
WALK

NOT D T
AND T J
WALK

AND D J
WALK

NOT D T
OR T J
WALK

OR D J
WALK

NOT A T
NOT T J
NOT B T
NOT T J
WALK

NOT A T
NOT T J
NOT B J
WALK

NOT A T
NOT T J
NOT B T
AND T J
WALK

NOT A T
NOT T J
AND B J
WALK

NOT A T
NOT T J
NOT B T
OR T J
WALK

NOT A T
NOT T J
OR B J
WALK

NOT A T
NOT T J
NOT C T
NOT T J
WALK

NOT A T
NOT T J
NOT C J
WALK

NOT A T
NOT T J
NOT C T
AND T J
WALK

NOT A T
NOT T J
AND C J
WALK

NOT A T
NOT T J
NOT C T
OR T J
WALK

NOT A T
NOT T J
OR C J
WALK

NOT A T
NOT T J
NOT D T
NOT T J
WALK

NOT A T
NOT T J
NOT D J
WALK

NOT A T
NOT T J
NOT D T
AND T J
WALK

NOT A T
NOT T J
AND D J
WALK

NOT A T
NOT T J
NOT D T
OR T J
WALK


AND C J
NOT A T
AND T J
WALK

AND C J
AND A J
WALK

AND C J
NOT A T
OR T J
WALK

AND C J
OR A J
WALK

AND C J
NOT B T
NOT T J
WALK

AND C J
NOT B J
WALK

AND C J
NOT B T
AND T J
WALK

AND C J
AND B J
WALK

AND C J
NOT B T
OR T J
WALK

AND C J
OR B J
WALK

AND C J
NOT D T
NOT T J
WALK

AND C J
NOT D J
WALK

AND C J
NOT D T
AND T J
WALK

AND C J
AND D J
WALK

AND C J
NOT D T
OR T J
WALK

AND C J
OR D J
WALK

NOT C T
OR T J
NOT A T
NOT T J
WALK

NOT C T
OR T J
NOT A J
WALK

NOT C T
OR T J
NOT A T
AND T J
WALK

NOT C T
OR T J
AND A J
WALK

NOT C T
OR T J
NOT A T
OR T J
WALK

NOT C T
OR T J
OR A J
WALK

NOT C T
OR T J
NOT B T
NOT T J
WALK

NOT C T
OR T J
NOT B J
WALK

NOT C T
OR T J
NOT B T
AND T J
WALK

NOT C T
OR T J
AND B J
WALK

NOT C T
OR T J
NOT B T
OR T J
WALK

NOT C T
OR T J
OR B J
WALK

NOT C T
OR T J
NOT D T
NOT T J
WALK

NOT C T
OR T J
NOT D J
WALK

NOT C T
OR T J
NOT D T
AND T J
WALK

NOT C T
OR T J
AND D J
WALK

NOT C T
OR T J
NOT D T
OR T J
WALK

NOT C T
OR T J
OR D J
WALK

OR

NOT A T
NOT T J
NOT C T
AND T J
NOT B T
NOT T J
WALK

NOT A T
NOT T J
NOT C T
AND T J
NOT B J
WALK

NOT A T
NOT T J
NOT C T
AND T J
NOT B T
AND T J
WALK

NOT A T
NOT T J
NOT C T
AND T J
AND B J
WALK

NOT A T
NOT T J
NOT C T
AND T J
NOT B T
OR T J
WALK

NOT A T
NOT T J
NOT C T
AND T J
OR B J
WALK

NOT A T
NOT T J
NOT C T
AND T J
NOT D T
NOT T J
WALK

NOT A T
NOT T J
NOT C T
AND T J
NOT D J
WALK

NOT A T
NOT T J
NOT C T
AND T J
NOT D T
AND T J
WALK

NOT A T
NOT T J
NOT C T
AND T J
AND D J
WALK

NOT A T
NOT T J
NOT C T
AND T J
NOT D T
OR T J
WALK

NOT A T
NOT T J
NOT C T
AND T J
OR D J
WALK

NOT A T
NOT T J
AND C J
NOT B T
NOT T J
WALK

NOT A T
NOT T J
AND C J
NOT B J
WALK

NOT A T
NOT T J
AND C J
NOT B T
AND T J
WALK

NOT A T
NOT T J
AND C J
AND B J
WALK

NOT A T
NOT T J
AND C J
NOT B T
OR T J
WALK

NOT A T
NOT T J
AND C J
OR B J
WALK

NOT A T
NOT T J
AND C J
NOT D T
NOT T J
WALK

NOT A T
NOT T J
AND C J
NOT D J
WALK

NOT A T
NOT T J
AND C J
NOT D T
AND T J
WALK

NOT A T
NOT T J
AND C J


NOT A J
NOT C T
AND T J
NOT B T
NOT T J
WALK

NOT A J
NOT C T
AND T J
NOT B J
WALK

NOT A J
NOT C T
AND T J
NOT B T
AND T J
WALK

NOT A J
NOT C T
AND T J
AND B J
WALK

NOT A J
NOT C T
AND T J
NOT B T
OR T J
WALK

NOT A J
NOT C T
AND T J
OR B J
WALK

NOT A J
NOT C T
AND T J
NOT D T
NOT T J
WALK

NOT A J
NOT C T
AND T J
NOT D J
WALK

NOT A J
NOT C T
AND T J
NOT D T
AND T J
WALK

NOT A J
NOT C T
AND T J
AND D J
WALK

NOT A J
NOT C T
AND T J
NOT D T
OR T J
WALK

NOT A J
NOT C T
AND T J
OR D J
WALK

NOT A J
AND C J
NOT B T
NOT T J
WALK

NOT A J
AND C J
NOT B J
WALK

NOT A J
AND C J
NOT B T
AND T J
WALK

NOT A J
AND C J
AND B J
WALK

NOT A J
AND C J
NOT B T
OR T J
WALK

NOT A J
AND C J
OR B J
WALK

NOT A J
AND C J
NOT D T
NOT T J
WALK

NOT A J
AND C J
NOT D J
WALK

NOT A J
AND C J
NOT D T
AND T J
WALK

NOT A J
AND C J
AND D J
WALK

NOT A J
AND C J
NOT D T
OR T J
WALK

NOT A J
AND C J
OR D J
WALK

NOT A J
NOT C T
OR T J
NOT B T
NOT T J
WALK

NOT A J
NOT C T
OR T J
NOT B J
WALK

NOT A J
NOT C 

([73, 110, 112, 117, 116, 32, 105, 110, 115, 116  …  107, 105, 110, 103, 46, 46, 46, 10, 10, 19354173], "NOT A J\nNOT C T\nOR T J\nAND D J\nWALK\n")

In [64]:
function floodfill(sensors = collect('A':'D'), move = "WALK\n")
    start = (Tuple{Char, Int}[], "")
    acc = [start]
    while !isempty(acc)
        s0, command = popfirst!(acc)
        used = [x[1] for x in s0]
        lng = isempty(s0) ? 0 : sum([mod(x[2], 2) + 1 for x in s0])
        avail = filter(x -> !(x in used), sensors)
        for sensor in avail
            if isempty(s0)
                iavail = 1:2
            else
                iavail = 3:6
            end
            for i in iavail
                println("Depth: ", lng + mod(i, 2))
                if lng + mod(i, 2) > 19 continue end
                new_command = command*sensor_command(sensor, i)*move
#                println(new_command)
                vm = Prog(readline("input.txt"))
                out = run(vm, Int.(collect(new_command)))
                if out[end] > 1000 return out, new_command end
                push!(acc, (vcat(s0, [(sensor, i)]), command*sensor_command(sensor, i)))
            end
        end
    end
    
    return [], "ERROR"
end

floodfill (generic function with 3 methods)

In [66]:
res = floodfill(collect('A':'I'), "RUN\n")

Depth: 1
Depth: 0
Depth: 1
Depth: 0
Depth: 1
Depth: 0
Depth: 1
Depth: 0
Depth: 1
Depth: 0
Depth: 1
Depth: 0
Depth: 1
Depth: 0
Depth: 1
Depth: 0
Depth: 1
Depth: 0
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 2
Depth: 1
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
D

Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 5
Depth: 4
Depth: 5
D

Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 3
Depth: 2
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
D

Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 5
Depth: 4
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
Depth: 3
Depth: 4
D

InterruptException: InterruptException: